In [1]:
#import xgboost as xgb
import pandas as pd
import numpy as np

### Прочитаем чищенные данные для первого отделения с добавленными фичами и добавим еще фич!

In [2]:
data = pd.read_csv("branch1_final_21_11.csv")

data["number_for_previous_6m"] = 0
data["half_a_year"] = 0
data["Quarter"] = 0
data.Cnt=round(data.Cnt,0)

data.half_a_year[data.Month<7]=1
data.half_a_year[data.Month>6]=2

data.Quarter[data.Month]=1
data.Quarter[data.Month>3]=2
data.Quarter[data.Month>6]=3
data.Quarter[data.Month>9]=4

C:\Users\Piboditheowl\Anaconda3\lib\site-packages\ipykernel\__main__.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Piboditheowl\Anaconda3\lib\site-packages\ipykernel\__main__.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Piboditheowl\Anaconda3\lib\site-packages\ipykernel\__main__.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Piboditheowl\Anaconda3\lib\site-packages\ipykernel\__main__.py:12: SettingWithCopyWarning: 
A value is trying to be set on 

### Подсчитаем сколько было посещений в полугодии и присвоим данные посещения следующему за ним. Для 2 полугодия 2013 года возьмем среднюю цифру, оценив насколько изменяется 1 полугодие и второе

In [3]:
k=0
polugod2 = np.zeros((1,3))
for i in range(2013,2016):
    x = 0
    x = sum(data.Cnt[data.Year==i][data.Month>6])
    polugod2[0,k]=x
    k+=1
k=0
polugod1 = np.zeros((1,2))
for i in range(2014,2016):
    x = 0
    x = x+sum(data.Cnt[data.Year==i][data.Month<7])
    polugod1[0,k]=x
    k+=1
data.number_for_previous_6m[(data.Year==2013) & (data.Month>6)]=44000
for i in range(0,2):    
    data.number_for_previous_6m[(data.Year==2014+i) & (data.Month>6)]=polugod1[0,i]
for i in range(0,2):    
    data.number_for_previous_6m[(data.Year==2014+i) & (data.Month<7)]=polugod2[0,i]


C:\Users\Piboditheowl\Anaconda3\lib\site-packages\ipykernel\__main__.py:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
C:\Users\Piboditheowl\Anaconda3\lib\site-packages\ipykernel\__main__.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [4]:
print (" Для первого полугодия 2016 цифра будет", polugod2[0,2])

 Для первого полугодия 2016 цифра будет 31972.0


In [5]:
data

,Cnt,Year,Month,Day,Hour,Day_of_week,number_for_previous_6m,half_a_year,Quarter
0,2.0,2013,10,1,10,2,44000,2,4
1,36.0,2013,10,1,11,2,44000,2,4
2,42.0,2013,10,1,12,2,44000,2,4
3,43.0,2013,10,1,13,2,44000,2,4
4,35.0,2013,10,1,14,2,44000,2,4
5,29.0,2013,10,1,15,2,44000,2,4
6,39.0,2013,10,1,16,2,44000,2,4
7,37.0,2013,10,1,17,2,44000,2,4
8,33.0,2013,10,1,18,2,44000,2,4
9,21.0,2013,10,1,19,2,44000,2,4


### Проверим модель на 2 полугодии 2015 года

In [21]:
test = data[data.Year==2015][data.Month>6]
train = data.drop(test.index)

/home/ven/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py:1: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  if __name__ == '__main__':


In [22]:
x_test = test.ix[:,1:9]
y_test = test.ix[:,0]
x_train = train.ix[:,1:9]
y_train = train.ix[:,0]
xgtrain = xgb.DMatrix(x_train, label=y_train)
xgtest = xgb.DMatrix(x_test, label=y_test)

In [42]:
watchlist = [(xgtrain, 'train'), (xgtest, 'eval')]
RANDOM_STATE = 42
params = {
            'min_child_weight': 1,
            'eta': 0.001,
            'colsample_bytree': 0.7,
            'max_depth': 5,
            'subsample': 0.7,
            'alpha': 1,
            'gamma': 1,
            'silent': 1,
            'verbose_eval': True,
            'seed': RANDOM_STATE,
            "print.every.n": 100,
            'objective': 'reg:linear'
    
        }

In [43]:
 model = xgb.train(params, xgtrain, 1000000, watchlist, early_stopping_rounds=300,maximize=False,verbose_eval = 2000)    

[0]	train-rmse:25.1852	eval-rmse:20.8287
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 300 rounds.
[2000]	train-rmse:8.44755	eval-rmse:8.24266
Stopping. Best iteration:
[2865]	train-rmse:7.39793	eval-rmse:7.98245



### eval-rmse:7.98245 для одного часа. Посмотрим относительную ошибку при предсказании нагрузки на все полугодие. Количество итераций 2865 .Для полной выборки должно быть порядка 3600

In [44]:
tolerance = 100*(sum(np.round(model.predict(xgtest),0)) - polugod2[0,2])/polugod2[0,2]
print ("Относительная ошибка предсказания для полугодия в целом составляет", abs(tolerance),"%")


Относительная ошибка предсказания для полугодия в целом составляет 2.186288002 %


### Обучим модель на полной выборке и оценим ошибку с помощью Кросс Валидации

In [26]:
x_data = data.ix[:,1:9]
y_data = data.ix[:,0]
xdata = xgb.DMatrix(x_data, label=y_data)

In [27]:
params = {
            'min_child_weight': 1,
            'eta': 0.001,
            'colsample_bytree': 0.7,
            'max_depth': 5,
            'subsample': 0.7,
            'alpha': 1,
            'gamma': 1,
            'silent': 1,
            'verbose_eval': True,
            'seed': RANDOM_STATE,
            "print.every.n": 100,
            'objective': 'reg:linear'
    
        }

In [28]:
model = xgb.cv(params, xdata, 3700, early_stopping_rounds=300,maximize=False,verbose_eval=300)    

[0]	train-rmse:24.3831+0.0624948	test-rmse:24.3829+0.124855
[300]	train-rmse:19.1366+0.0487522	test-rmse:19.1488+0.126879
[600]	train-rmse:15.3731+0.0444122	test-rmse:15.4021+0.124236
[900]	train-rmse:12.7359+0.0408482	test-rmse:12.7831+0.129173
[1200]	train-rmse:10.9091+0.0380299	test-rmse:10.9785+0.136757
[1500]	train-rmse:9.66153+0.0347073	test-rmse:9.75311+0.145143
[1800]	train-rmse:8.81668+0.0338979	test-rmse:8.93196+0.148623
[2100]	train-rmse:8.24106+0.0297385	test-rmse:8.37967+0.15194
[2400]	train-rmse:7.8463+0.028768	test-rmse:8.00753+0.150401
[2700]	train-rmse:7.56466+0.0268357	test-rmse:7.74617+0.147786
[3000]	train-rmse:7.35793+0.0250929	test-rmse:7.5601+0.143674
[3300]	train-rmse:7.20052+0.0252963	test-rmse:7.42119+0.139262
[3600]	train-rmse:7.07762+0.0274689	test-rmse:7.31666+0.131693


### Точность улучшились test-rmse:7.31666 Что связано как с увеличением числа наблюдений, так и с самой методикой оценки. Количество итераций = 3700. При кросс валидации можно получить RMSE порядка 5 но есть риск переобучения

In [45]:
model = xgb.train(params, xdata, 3600)    

### Проверим. Количество посещений за 2 полугодие 2015 было 31972 - оно будет использовано для предсказания 1 полугодия 2016

In [46]:
x = np.array([[2016,1,11,8,1,31972,1,1],[2016,1,11,9,1,31972,1,1],[2016,1,11,10,1,31972,1,1],[2016,1,11,11,1,31972,1,1]]) # задаем год = 2016, месяц - 1(январь), день месяца 11, час - 10, день недели 
# понедельник - 1 , количество посещений за предыдущее полугодие 25437.0, полугодие -1, квартал - 1

In [47]:
x = pd.DataFrame(x)
x.columns = x_data.columns
proverka = xgb.DMatrix(x)
pr = np.round(model.predict(proverka),0)
print('11 Января 2016 года в 8 утра придет', pr[0], ',в 9 утра', pr[1], ',в 10 утра', pr[2], ',в 11 утра', pr[3])

11 Января 2016 года в 8 утра придет 8.0 ,в 9 утра 16.0 ,в 10 утра 23.0 ,в 11 утра 25.0


In [48]:
model.save_model("bst.model")
